In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string as st
import re
import warnings
warnings.filterwarnings("ignore")
import spacy
en = spacy.load('en_core_web_sm')
import string

In [109]:
df = pd.read_csv('/content/train - train.csv')

In [110]:
def lower_case(df):
  df['aspect']= df['aspect'].apply(lambda x: x.lower())
  df['text']= df['text'].apply(lambda x: x.lower())
  return df

In [111]:
lower_case(df)

,text,aspect,label
0,can you check whether its cancelled completely?,cancelled,1
1,cannot rely on both milk delivery and grocery ...,milk,0
2,"i get no notification, however the app is real...",notification,0
3,"love this app, but would love it even more if ...",view,1
4,it does not let me load a clip on the scene,load,0
...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0
3996,unable to load money using wallets (phonepe/ol...,unable to load money using wallets,0
3997,"hi, i m doing for the first time ever and i no...",usage,0
3998,delivery is delayed or cancelled every time i ...,delivery,0


In [112]:
def return_clauses(text):
  doc = en(text)
#deplacy.render(doc)

  seen = set() # keep track of covered words

  chunks = []
  for sent in doc.sents:
      heads = [cc for cc in sent.root.children if cc.dep_ == 'conj']

      for head in heads:
          words = [ww for ww in head.subtree]
          for word in words:
              seen.add(word)
          chunk = (' '.join([ww.text for ww in words]))
          chunks.append( (head.i, chunk) )

      unseen = [ww for ww in sent if ww not in seen]
      chunk = ' '.join([ww.text for ww in unseen])
      chunks.append( (sent.root.i, chunk) )

  chunks = sorted(chunks, key=lambda x: x[0])
  list_of_clauses = []
  for ii, chunk in chunks:
      list_of_clauses.append(chunk)
      #print(chunk)
  return(list_of_clauses)


In [113]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

df['text']= df['text'].apply(lambda x:remove_punctuation(x))
df['aspect']= df['aspect'].apply(lambda x:remove_punctuation(x))

In [114]:
def split_using_aspect(data):
  df['new_text']= df['text'].apply(lambda x:return_clauses(x))

In [115]:
split_using_aspect(df)

In [116]:
result = []
def find_aspect(df):
  for index in range(len(df)):
    #print(df['aspect'][index],df['new_text'][index])
    res = [i for i in df['new_text'][index] if df['aspect'][index] in i]
    result.append(res)
  #print(len(result))
  df['new_column'] = result
  return df

In [117]:
find_aspect(df)

,text,aspect,label,new_text,new_column
0,can you check whether its cancelled completely,cancelled,1,[can you check whether its cancelled completely],[can you check whether its cancelled completely]
1,cannot rely on both milk delivery and grocery ...,milk,0,[can not rely on both milk delivery and grocer...,[can not rely on both milk delivery and grocer...
2,i get no notification however the app is reall...,notification,0,[i get no notification however the app is real...,[i get no notification however the app is real...
3,love this app but would love it even more if g...,view,1,"[love this app but, would love it even more if...",[would love it even more if gantt charts and c...
4,it does not let me load a clip on the scene,load,0,[it does not let me load a clip on the scene],[it does not let me load a clip on the scene]
...,...,...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0,[every time i try to edit a page or create a l...,[every time i try to edit a page or create a l...
3996,unable to load money using wallets phonepeola ...,unable to load money using wallets,0,[unable to load money using wallets phonepeola...,[unable to load money using wallets phonepeola...
3997,hi i m doing for the first time ever and i not...,usage,0,"[hi i m doing for the first time ever, and i n...",[and i not able to understand the usage of all...
3998,delivery is delayed or cancelled every time i ...,delivery,0,"[delivery is delayed or, cancelled every time ...",[delivery is delayed or]
